In [1]:
import networkx as nx
from networkx.readwrite import json_graph
import json
import sys
import string
import random
import numpy as np
import itertools

In [2]:
# generate random label
def generate_random_label():
    base = ''.join(random.choice(string.ascii_uppercase) for i in range(2))
    n1 = int(random.random()*10)
    n2 = int(random.random()*10)
    return base + str(n1) + str(n2)

In [3]:
# network settings
# nodes = random.randint(35, 45) #[35, 45] rng
nodes = 6
selected_set = set()
clique = True # False True
print(nodes)

6


In [4]:
# generate scale free graph
G = nx.scale_free_graph(nodes, 0.48, 0.45, 0.07)
# alpha (float) – Probability for adding a new node connected to an existing node chosen randomly according to the in-degree distribution.
# beta (float) – Probability for adding an edge between two existing nodes. One existing node is chosen randomly according the in-degree distribution and the other chosen randomly according to the out-degree distribution.
# gamma (float) – Probability for adding a new node connected to an existing node chosen randomly according to the out-degree distribution.

In [5]:
# remove self loops
for s, t, k in list(G.edges(data=True)):
    if s == t: 
        G.remove_edge(s,t)

In [6]:
# remove multi edges
unique_edges = dict()
for s, t, k in list(G.edges(data=True)):
    if f'{s}:{t}' in unique_edges: 
        unique_edges[f'{s}:{t}'] = unique_edges.get(f'{s}:{t}') + 1
    else: 
        unique_edges[f'{s}:{t}'] = 1
        
for s, t, k in list(G.edges(data=True)):
    if f'{s}:{t}' in unique_edges and unique_edges.get(f'{s}:{t}') > 1:
        G.remove_edge(s,t)
        unique_edges[f'{s}:{t}'] = unique_edges.get(f'{s}:{t}') - 1

In [269]:
# generate clique
if clique:
    number_of_nodes = 5
    nodes = random.sample(G.nodes, number_of_nodes)
    permutations = list(itertools.permutations(nodes, 2))
    print(f'{number_of_nodes} nodes selected at random:')
    print(nodes)
    print('All permutations:')
    print(permutations)
    for i in permutations:
        source = i[0]
        target = i[1]
        if (G.has_edge(source, target)) or (G.has_edge(target, source)): 
            print(f'{source}-{target} or {target}-{source} exists')
            continue
        else: 
            print(f'{source}-{target} or {target}-{source} does not exist --- adding to edges')
            G.add_edge(source, target) 

5 nodes selected at random:
[34, 22, 4, 14, 6]
All permutations:
[(34, 22), (34, 4), (34, 14), (34, 6), (22, 34), (22, 4), (22, 14), (22, 6), (4, 34), (4, 22), (4, 14), (4, 6), (14, 34), (14, 22), (14, 4), (14, 6), (6, 34), (6, 22), (6, 4), (6, 14)]
34-22 or 22-34 does not exist --- adding to edges
34-4 or 4-34 does not exist --- adding to edges
34-14 or 14-34 does not exist --- adding to edges
34-6 or 6-34 does not exist --- adding to edges
22-34 or 34-22 exists
22-4 or 4-22 does not exist --- adding to edges
22-14 or 14-22 does not exist --- adding to edges
22-6 or 6-22 does not exist --- adding to edges
4-34 or 34-4 exists
4-22 or 22-4 exists
4-14 or 14-4 does not exist --- adding to edges
4-6 or 6-4 does not exist --- adding to edges
14-34 or 34-14 exists
14-22 or 22-14 exists
14-4 or 4-14 exists
14-6 or 6-14 does not exist --- adding to edges
6-34 or 34-6 exists
6-22 or 22-6 exists
6-4 or 4-6 exists
6-14 or 14-6 exists


In [7]:
# assign labels to nodes
print("Created graph with " + str(len(G.nodes)) + " nodes and " + str(len(G.edges)) + " edges.")
for i in range(len(G.nodes)):
    label = generate_random_label()
    while label in selected_set:
        label = generate_random_label()
    selected_set.add(label)
    G.nodes[i]["label"] = label

Created graph with 6 nodes and 6 edges.


In [8]:
# assign time steps to edges
# pump up probability of disappearance
for s, t, k in list(G.edges(data=True)):
    timeArray = [int(np.random.choice(np.arange(0, 2, 1), p = [0.1, 0.9])),
                 int(np.random.choice(np.arange(0, 2, 1), p = [0.1, 0.9])),
                 int(np.random.choice(np.arange(0, 2, 1), p = [0.1, 0.9])),
                 int(np.random.choice(np.arange(0, 2, 1), p = [0.1, 0.9]))
                ]
    G.adj[s][t][0]['time'] = timeArray

In [9]:
# generate file name
file_name = "SFG_"+ str(len(G.nodes)) + "_" + str(len(G.edges))

if clique:
    file_name += "_clique"

In [10]:
# write out file as json
json_data = nx.json_graph.node_link_data(G)

with open(file_name + '.json', 'w') as outfile:
    json.dump(json_data, outfile)

print(file_name, "done.")

SFG_6_6_clique done.
